In [2]:
!unzip ../model/fine_tuned_with_cs.zip

Archive:  ../model/fine_tuned_with_cs.zip
   creating: llama3.2-3b-qlora-summary
  inflating: llama3.2-3b-qlora-summary/README.md  
   creating: llama3.2-3b-qlora-summary/checkpoint-100
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/README.md  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/adapter_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/training_args.bin  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/special_tokens_map.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/tokenizer_config.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/rng_state.pth  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/trainer_state.json  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/chat_template.jinja  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/optimizer.pt  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/scheduler.pt  
  inflating: llama3.2-3b-qlora-summary/checkpoint-100/adapter_model.safetensors  

In [ ]:
!unzip final-summary.zip

In [2]:
!pip install pandas datasets


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install transformers torch

In [ ]:
!pip install xformers trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 46.1 MB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 20.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for xformers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [247 lines of output]
      /private/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/pip-build-env-lx2rtou5/overlay/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
        cpu = _conversion_method_template(device=torch.device("cpu"))
      /private/var/folders/5z/ksmd9z810hv7hd63k2mmtgzr0000gn/T/pip-build-env-lx2rtou5/overlay/lib/python3.13/site-packages/se

In [1]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
from gc import collect
from pathlib import Path

/Users/thorkildkappel/Desktop/7. sem/NLP /project/NLP_project_group6/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("We would be using this device:", device)

We would be using this device: mps


In [2]:
# Load JSONL data (Custom Dataset)
print("\n2-[1/8] Loading dataset...")
data = []
with open("../data/custom_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
val_dataset = split_dataset["test"]

print(f"✓ Loaded {len(val_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 101 examples


In [3]:
def extract_msg_for_base_model(_msg):
    messages = _msg["messages"]
    return {
        "messages": [
            {
                "role": "system",
                "content": """You are an AI assistant that helps provide the summary for a youtube video given we provide you with a user search text, and based on the search text youtube has given out 4 relevant videos as a result, we will give you the search text, video title and the transcript of all the result videos (it could be between 1 to 4 ), your job is to summarize the video according to the prompt and also address the similarities and differences in the video's opinion over the user's query.

for example:

SEARCH_TEXT: Whats the Best thing to do on Holidays

VIDEO_1_TITLE: Top 5 things to do on Holidays
VIDEO_1_TRANSCRIPT: One of the Thing to do on Holiday is to visit your grandparents and spend some time with your family....

VIDEO_2_TITLE: Top 5 Games to try on Holidays
VIDEO_2_TRANSCRIPT: Well you can try Death Stranding or Far Cry (Entire Series) so that u have time to play and also see how they grow over time and you won't see how time passes so quickly...

VIDEO_3_TITLE: How to Convince your boss to work on Holidays
VIDEO_3_TRANSCRIPT: Tells ways to work on Holidays instead of Enjoying...

I am expecting Output like...

Key points from Video_1:
...

Key points from Video_2:
...

Transcript 2 unpacked:
...

well Video 1 and Video 2 agree on the fact that you should spend some time in leisure during holiday be it playing games or with your family. Video 1 seems to be giving good advice and Suggestions from Video 2 can also be considered if you are into games, while Video 3 out of no where suggesting things that might not be average thing to do on Holidays, it focuses more on Working than Enjoying as opposed to  other videos


That's the Example, please make sure to follow these rules as well.
- No bullet points, no lists, no headings.
- Do not use any characters that cannot be typed directly on a standard keyboard, including but not limited to em-dashes (-), en-dashes (-), smart quotes (" ", ' '), or any other special typographic symbols. Use only standard ASCII characters.
- You may also compare videos, for example: "video 1 is similar to video 2 except for this part where they disagree", or "video 3 is more similar to video 1 where the speakers have the same views on this...".
- Combine the relevant points from the transcripts into smooth, continuous paragraph(s) for each video.
- Ignore irrelevant or repeated transcript content.
- Tone should be clear, helpful, and natural.
- Always reference videos by their index like VIDEO_1, VIDEO_2.
- For each video, write a short section summarizing only the key insights. Keep it 3–5 sentences or around 50–100 words.
"""
            },
            {
                "role": "user",
                "content": messages[1]["content"]
            }
        ]
    }

In [4]:
base_generation_inputs = val_dataset.map(extract_msg_for_base_model)

Map: 100%|██████████| 101/101 [00:00<00:00, 5733.72 examples/s]


In [5]:
BASE_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

In [6]:
bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.bfloat16,
    )

In [13]:
!huggingface-cli login


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ^C
Traceback (most recent call last):
  File "/Users/thorkildkappel/Desktop/7. sem/NLP /project/NLP_project_group6/.venv/bin/huggingface-cli", line 10, in <module>
    sys.exit(main())
  File "/

In [14]:
base_tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL,
        use_fast=True
    )
# LLaMA models usually do not have a pad token by default
if base_tokenizer.pad_token is None:
    base_tokenizer.pad_token = base_tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
)


KeyboardInterrupt: 

In [ ]:
base_model.eval()
base_outputs = []
for base_msg in base_generation_inputs:
    prompt = base_tokenizer.apply_chat_template(
        base_msg["messages"],
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = base_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=base_tokenizer.model_max_length,
    ).to(base_model.device)


    with torch.no_grad():
        output = base_model.generate(
            **inputs,
            max_new_tokens=10_000,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            eos_token_id=base_tokenizer.eos_token_id,
            pad_token_id=base_tokenizer.eos_token_id,
        )

    response = base_tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    base_outputs.append((base_msg, response))
    print(len(response))


4472
2173
2588
3493
1333
7101
3200
2747
3450
2684
2382
5195
3245
2329
3437
1777
3072
1929
2397
3252
2971
3297
2854
2745
2402
3726
1992
2768
1412
3116
3023
2304
5197
3501
3816
2858
2933
1648
2644
3583
3513
2842
2248
2041
2094
1816
4051
1813
2608
3987


In [ ]:
len(base_outputs)

In [ ]:
Path("base_outputs.json").write_text(json.dumps(base_outputs))

In [ ]:
del base_model
del base_tokenizer
collect()

In [7]:
ADAPTER_PATH = "./llama3.2-3b-qlora-summary"

In [8]:
def split_msgs(_msg):
    messages = _msg["messages"]
    return {
        "messages": messages[: 2]
    }

In [9]:
generation_inputs = val_dataset.map(split_msgs)

Map: 100%|██████████| 101/101 [00:00<00:00, 7103.39 examples/s]


In [10]:
cs_tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)


cs_tokenizer.pad_token = cs_tokenizer.eos_token

cs_tokenizer.padding_side = "right"

cs_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,  # SAME as original
)

cs_model = prepare_model_for_kbit_training(cs_model, use_gradient_checkpointing=True)
cs_model = PeftModel.from_pretrained(
    cs_model,
    ADAPTER_PATH,
    is_trainable=True   # to continue training
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.91s/it]


In [14]:
cs_model.eval()
cs_outputs = []
total = len(generation_inputs)
for index, cs_msg in enumerate(generation_inputs):
    prompt = cs_tokenizer.apply_chat_template(
        cs_msg["messages"],
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = cs_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=cs_tokenizer.model_max_length,
    ).to(cs_model.device)

    print("generating")

    with torch.no_grad():
        output = cs_model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            eos_token_id=cs_tokenizer.eos_token_id,
            pad_token_id=cs_tokenizer.eos_token_id,
        )

    print("decoding")
    response = cs_tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    cs_outputs.append((cs_msg, response))
    print(f"{((index + 1) / total) * 100}% - {len(response)}")


generating


/Users/thorkildkappel/Desktop/7. sem/NLP /project/NLP_project_group6/.venv/lib/python3.13/site-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


KeyboardInterrupt: 

In [ ]:
import pandas as pd

df = pd.DataFrame(cs_outputs, columns=['cs_msg', 'response'])

df.to_csv()